In [2]:
import pandas as pd
from backtesting import Backtest, Strategy
from backtesting.lib import crossover

# Simple Moving Average function
def SMA(values, n):
    """
    Return simple moving average of `values`, at
    each step taking into account `n` previous values.
    """
    return pd.Series(values).rolling(n).mean()

# SMA Cross Strategy class
class SmaCross(Strategy):
    n1 = 30
    n2 = 100

    def init(self):
        # Precompute the two moving averages
        self.sma1 = self.I(SMA, self.data.Close, self.n1)
        self.sma2 = self.I(SMA, self.data.Close, self.n2)

    def next(self):
        # If sma1 crosses above sma2, close any existing
        # short trades, and buy the asset
        if crossover(self.sma1, self.sma2):
            self.position.close()
            self.buy()

        # Else, if sma1 crosses below sma2, close any existing
        # long trades, and sell the asset
        elif crossover(self.sma2, self.sma1):
            self.position.close()
            self.sell()



/Users/liantseng/Desktop/python-backtest-Demo/venv/lib/python3.13/site-packages/backtesting/_plotting.py:55: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support, such as old IDEs. Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

In [4]:
import yfinance as yf

# Get BTC-USD data from yfinance
data = yf.download('BTC-USD')
if isinstance(data.columns, pd.MultiIndex):
    data.columns = data.columns.droplevel(1)  # 移除第二層索引(ticker名稱)
# Code for running the backtest.
# Assumes the existence of `data` variable containing backtest data.
bt = Backtest(data, SmaCross)
stats = bt.run()


YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed
/var/folders/3x/gvlk_pqx3px5mcf8byl_s4vw0000gn/T/ipykernel_20768/1662273915.py:9: UserWarning: Some prices are larger than initial cash value. Note that fractional trading is not supported. If you want to trade Bitcoin, increase initial cash, or trade μBTC or satoshis instead (GH-134).
  bt = Backtest(data, SmaCross)


In [5]:
#print(stats['_trades'].to_string())
stats


Start                     2014-09-17 00:00:00
End                       2025-06-05 00:00:00
Duration                   3914 days 00:00:00
Exposure Time [%]                    94.55939
Equity Final [$]                 611728.87241
Equity Peak [$]                  861468.18491
Return [%]                         6017.28872
Buy & Hold Return [%]             32709.03267
Return (Ann.) [%]                    46.74488
Volatility (Ann.) [%]               122.60729
CAGR [%]                             46.75926
Sharpe Ratio                          0.38126
Sortino Ratio                         0.97865
Calmar Ratio                           0.4961
Alpha [%]                         -4427.12084
Beta                                  0.31931
Max. Drawdown [%]                    -94.2255
Avg. Drawdown [%]                   -13.85563
Max. Drawdown Duration      727 days 00:00:00
Avg. Drawdown Duration       66 days 00:00:00
# Trades                                   34
Win Rate [%]                      

## 基本時間信息

- __Start__: 2014-09-17 00:00:00 - 回測開始日期
- __End__: 2025-06-04 00:00:00 - 回測結束日期
- __Duration__: 3913 days 00:00:00 - 回測總時長（約10.7年）

## 收益與風險指標

- __Exposure Time [%]__: 94.60772 - 資金在市場中的暴露時間百分比，表示有94.6%的時間有持倉
- **Equity Final []∗∗:616413.79428−最終資金（假設初始資金為100,000）
- __Equity Peak [$]__: 861468.18491 - 資金最高峰值
- __Return [%]__: 6064.13794 - 總回報率（約60.6倍）
- __Buy & Hold Return [%]__: 33002.75082 - 買入並持有策略的回報率（約330倍），這表明簡單持有比您的策略表現更好
- __Return (Ann.) [%]__: 46.87814 - 年化回報率
- __Volatility (Ann.) [%]__: 122.7576 - 年化波動率，表明策略風險較高
- __CAGR [%]__: 46.87814 - 複合年增長率，與年化回報率相同

## 風險調整回報指標

- __Sharpe Ratio__: 0.38188 - 夏普比率（回報與風險的比值），低於1表示風險調整後回報不佳
- __Sortino Ratio__: 0.9812 - 索提諾比率（只考慮下行風險），接近1表示相對較好
- __Calmar Ratio__: 0.49751 - 卡爾馬比率（年化回報與最大回撤的比值），低於1表示風險較高
- __Alpha [%]__: -4473.77617 - 相對於買入持有策略的超額收益，負值表示表現不如基準
- __Beta__: 0.3193 - 相對於市場的波動性，低於1表示波動性低於市場

## 回撤指標

- __Max. Drawdown [%]__: -94.2255 - 最大回撤（從峰值到谷值的最大損失），非常高
- __Avg. Drawdown [%]__: -13.85563 - 平均回撤
- __Max. Drawdown Duration__: 727 days 00:00:00 - 最長回撤持續時間（約2年）
- __Avg. Drawdown Duration__: 66 days 00:00:00 - 平均回撤持續時間

## 交易統計

- __# Trades__: 34 - 總交易次數
- __Win Rate [%]__: 47.05882 - 勝率（約47%）
- __Best Trade [%]__: 1113.5341 - 最佳交易收益（約11倍）
- __Worst Trade [%]__: -64.72543 - 最差交易損失
- __Avg. Trade [%]__: 13.2215 - 平均每筆交易收益
- __Max. Trade Duration__: 479 days 00:00:00 - 最長持倉時間（約1.3年）
- __Avg. Trade Duration__: 109 days 00:00:00 - 平均持倉時間

## 高級指標

- __Profit Factor__: 6.53027 - 總盈利除以總虧損，大於1表示策略有利可圖
- __Expectancy [%]__: 51.08753 - 每筆交易的期望收益
- __SQN__: 0.96958 - 系統質量數（System Quality Number），接近1表示系統質量一般
- __Kelly Criterion__: 0.19794 - 凱利準則，建議的最佳資金分配比例

## 其他信息

- ___strategy__: SmaCross - 使用的策略名稱
- ___equity_curve__: ... - 資金曲線數據（被截斷）
- ___trades__: Size EntryB... - 交易詳情（被截斷）


In [6]:
# Define a range of values to test for each parameter
param_grid = {'n1': range(5, 60, 5), 'n2': range(10, 90, 5)}
# Run the optimization
res = bt.optimize(**param_grid)

# Print the best results and the parameters that lead to these results
print("Best result: ", res['Return [%]'])
print("Parameters for best result: ", res['_strategy'])

/Users/liantseng/Desktop/python-backtest-Demo/venv/lib/python3.13/site-packages/backtesting/backtesting.py:1606: RuntimeWarning: If you want to use multi-process optimization with `multiprocessing.get_start_method() == 'spawn'` (e.g. on Windows),set `backtesting.Pool = multiprocessing.Pool` (or of the desired context) and hide `bt.optimize()` call behind a `if __name__ == '__main__'` guard. Currently using thread-based paralellism, which might be slightly slower for non-numpy / non-GIL-releasing code. See https://github.com/kernc/backtesting.py/issues/1256
  output = _optimize_grid()
/Users/liantseng/Desktop/python-backtest-Demo/venv/lib/python3.13/site-packages/backtesting/backtesting.py:954: UserWarning: time=3534: Broker canceled the relative-sized order due to insufficient margin.
  warnings.warn(
/Users/liantseng/Desktop/python-backtest-Demo/venv/lib/python3.13/site-packages/backtesting/backtesting.py:954: UserWarning: time=2144: Broker canceled the relative-sized order due to ins

Best result:  32665.19115493774
Parameters for best result:  SmaCross(n1=10,n2=35)


In [7]:
res

Start                     2014-09-17 00:00:00
End                       2025-06-05 00:00:00
Duration                   3914 days 00:00:00
Exposure Time [%]                    97.82886
Equity Final [$]                3276519.11549
Equity Peak [$]                 3493824.27174
Return [%]                        32665.19115
Buy & Hold Return [%]             26981.55401
Return (Ann.) [%]                    71.59904
Volatility (Ann.) [%]               115.65536
CAGR [%]                             71.62271
Sharpe Ratio                          0.61907
Sortino Ratio                         1.78267
Calmar Ratio                           1.3144
Alpha [%]                         29677.46871
Beta                                  0.11073
Max. Drawdown [%]                   -54.47261
Avg. Drawdown [%]                   -10.57834
Max. Drawdown Duration      962 days 00:00:00
Avg. Drawdown Duration       46 days 00:00:00
# Trades                                  115
Win Rate [%]                      

In [8]:
res['_equity_curve']

,Equity,DrawdownPct,DrawdownDuration
Date,,,
2014-09-17,1.000000e+04,0.000000,NaT
2014-09-18,1.000000e+04,0.000000,NaT
2014-09-19,1.000000e+04,0.000000,NaT
2014-09-20,1.000000e+04,0.000000,NaT
2014-09-21,1.000000e+04,0.000000,NaT
...,...,...,...
2025-06-01,3.307168e+06,0.053425,NaT
2025-06-02,3.314280e+06,0.051389,NaT
2025-06-03,3.300359e+06,0.055373,NaT


顯示所有交易歷史

In [9]:
res['_trades']

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,SL,TP,PnL,ReturnPct,EntryTime,ExitTime,Duration,Tag,"Entry_SMA(C,10)","Exit_SMA(C,10)","Entry_SMA(C,35)","Exit_SMA(C,35)"
0,-28,39,58,347.487000,418.416992,None,None,-1986.039795,-0.204123,2014-10-26,2014-11-14,19 days,None,373.596097,371.667603,376.430000,365.760258
1,19,58,85,418.416992,344.339996,None,None,-1407.462921,-0.177041,2014-11-14,2014-12-11,27 days,None,371.667603,366.366202,365.760258,372.212514
2,-19,85,153,344.339996,233.421997,None,None,2107.441986,0.322118,2014-12-11,2015-02-17,68 days,None,366.366202,230.933603,372.212514,227.732001
3,37,153,190,233.421997,246.276001,None,None,475.598145,0.055068,2015-02-17,2015-03-26,37 days,None,230.933603,259.953000,227.732001,265.426401
4,-37,190,234,246.276001,243.768997,None,None,92.759140,0.010180,2015-03-26,2015-05-09,44 days,None,259.953000,237.146201,265.426401,234.050486
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
110,-29,3754,3778,95704.976562,101083.750000,None,None,-155984.429688,-0.056202,2024-12-27,2025-01-20,24 days,None,97023.999219,99234.338281,98195.975223,97448.160045
111,26,3778,3797,101083.750000,96533.257812,None,None,-118312.796875,-0.045017,2025-01-20,2025-02-08,19 days,None,99234.338281,99098.273438,97448.160045,99884.868527
112,-26,3797,3850,96533.257812,85180.609375,None,None,295168.859375,0.117603,2025-02-08,2025-04-02,53 days,None,99098.273438,84853.765625,99884.868527,84973.928571
113,33,3850,3851,85180.609375,82487.476562,None,None,-88873.382812,-0.031617,2025-04-02,2025-04-03,1 days,None,84853.765625,84414.157031,84973.928571,84928.174330
